In [2]:
import math
import pandas as pd

from sklearn.metrics import confusion_matrix, f1_score

In [3]:
words = pd.read_csv('../data/frequencies/statistical_phrases.csv', index_col=0)
words.head(2)

,NEUTRAL,GOOD,BAD,all_count,word_counts
word,,,,,
в,0.99,1.03,0.99,23644,1
и,1.00,1.05,0.92,16356,1


In [4]:
words_dict = {word: info.to_dict() for word, info in words.iterrows()}

In [5]:
normalize_texts = pd.read_csv('../data/marks_csv/aggregated_normalize_phrases.csv', index_col=None)
normalize_texts.head(2)

,input_normal_form_phrases,output,confidence
0,условие термодинамический согласованность имет...,NEUTRAL,0.92
1,мишень подтвердить там для модель дополнить пр...,NEUTRAL,0.90


In [7]:
def get_correct_value(values: dict, name: str) -> float:
    value = values.get(name)
    return value if not pd.isna(value) else 0

results = []
for _, row in normalize_texts.iterrows():
    weights = {'NEUTRAL': 0., 'GOOD': 0., 'BAD': 0.}
    for word in row['input_normal_form_phrases'].split():
        word_info = words_dict.get(word, {})
        weights['NEUTRAL'] += get_correct_value(word_info, 'NEUTRAL')
        weights['GOOD'] += get_correct_value(word_info, 'GOOD')
        weights['BAD'] += get_correct_value(word_info, 'BAD')
    results.append(dict(output=row['output'], predict=sorted(weights.items(), key=lambda x: x[1])[-1][0], **weights))

results = pd.DataFrame(results)
results.head(2)

,BAD,GOOD,NEUTRAL,output,predict
0,72.63,62.77,69.09,NEUTRAL,BAD
1,86.38,81.43,84.49,NEUTRAL,BAD


In [8]:
confusion_matrix(results['output'].tolist(), results['predict'].tolist(), labels=["GOOD", "NEUTRAL", "BAD"])

array([[1868,   72,  291],
       [ 685, 2367,  843],
       [  68,   39, 1527]], dtype=int64)

In [9]:
(
    f1_score(results['output'].tolist(), results['predict'].tolist(), labels=["GOOD", "NEUTRAL", "BAD"], average='weighted'),
    f1_score(results['output'].tolist(), results['predict'].tolist(), labels=["GOOD", "NEUTRAL", "BAD"], average='micro'),
    f1_score(results['output'].tolist(), results['predict'].tolist(), labels=["GOOD", "NEUTRAL", "BAD"], average='macro'),
)

(0.7439447803261917, 0.7425257731958763, 0.7412908015341403)

In [13]:
results[(results['output'] == 'GOOD') & (results['predict'] == 'BAD')].head(5)

,BAD,GOOD,NEUTRAL,output,predict
14,65.34,64.13,61.32,GOOD,BAD
161,71.05,67.27,61.16,GOOD,BAD
186,49.57,48.06,47.31,GOOD,BAD
306,71.90,68.29,50.26,GOOD,BAD
327,69.87,68.13,53.19,GOOD,BAD


In [16]:
normalize_texts.loc[14].input_normal_form_phrases    

'аналогичный результат быть получить в эксперимент в который расстояние перенос составлять а число нуклеотидный пара разделять донор и акцептор равняться совершенно неожиданный оказаться результат эксперимент в который изучаться перенос электрон на больший расстояние что составлять нуклеотидный пара разделять донор и акцептор несмотря на то что расстояние перенос в эксперимент почти вдвое большой чем в скорость перенос получить в оказаться на три порядок выше'